In [1]:
import collections.abc

# Define the M4System class to hold the parameters and modulus for the 4D operation
# This encapsulates the specific algebraic system (V=K^4, *)
class M4System:
    def __init__(self, A, B, C, D, E, F, G, H, I, modulus):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        self.E = E
        self.F = F
        self.G = G
        self.H = H
        self.I = I
        self.modulus = modulus

    # Override __repr__ for better readability of the system object
    def __repr__(self):
        return (f"M4System(A={self.A}, B={self.B}, C={self.C}, D={self.D}, E={self.E}, "
                f"F={self.F}, G={self.G}, H={self.H}, I={self.I}, modulus={self.modulus})")

# Define the M4Element class to represent vectors within a specific M4System
class M4Element:
    def __init__(self, value: list[int], system: M4System):
        if not isinstance(value, collections.abc.Sequence) or len(value) != 4:
            raise ValueError("Value must be a list or tuple of 4 integers.")
        if not isinstance(system, M4System):
            raise TypeError("System must be an instance of M4System.")

        self.system = system
        self.value = [x % self.system.modulus for x in value]

    # Standard vector addition for the underlying vector space
    def __add__(self, other):
        if not isinstance(other, M4Element) or self.system != other.system:
            return NotImplemented # Or raise ValueError for system mismatch
        return M4Element([(x + y) % self.system.modulus for x, y in zip(self.value, other.value)], self.system)

    # Standard vector subtraction
    def __sub__(self, other):
        if not isinstance(other, M4Element) or self.system != other.system:
            return NotImplemented
        return M4Element([(x - y) % self.system.modulus for x, y in zip(self.value, other.value)], self.system)

    # Standard unary negation
    def __neg__(self):
        return M4Element([(-x) % self.system.modulus for x in self.value], self.system)

    # The core binary operation '*' as defined in the article for K^4
    # Corresponds to (ab) in the article
    def __mul__(self, other):
        if not isinstance(other, M4Element) or self.system != other.system:
            return NotImplemented # Or raise ValueError for system mismatch

        # Components of vector 'a' (self)
        a0, a1, a2, a3 = self.value
        # Components of vector 'b' (other)
        b0, b1, b2, b3 = other.value
        
        # Parameters of the M4System
        A, B, C, D, E, F, G, H, I = (self.system.A, self.system.B, self.system.C, self.system.D,
                                      self.system.E, self.system.F, self.system.G, self.system.H, self.system.I)
        N = self.system.modulus

        # Component-wise definition of (ab)_i based on the article's K^4 formula
        # (ab)_0 = a_0 + b_0 + a_0 b_0 + A a_1 b_1 + E a_3 b_1 + B a_2 b_2 + D a_1 b_2 + F a_3 b_2 + C a_3 b_3
        r0 = (a0 + b0 + a0 * b0 + A * a1 * b1 + E * a3 * b1 + 
              B * a2 * b2 + D * a1 * b2 + F * a3 * b2 + C * a3 * b3) % N
        
        # (ab)_1 = a_1 + b_1 + a_1 b_0 + a_0 b_1 + G a_1 b_1 + H a_1 b_2 + I a_1 b_3
        r1 = (a1 + b1 + a1 * b0 + a0 * b1 + G * a1 * b1 + H * a1 * b2 + I * a1 * b3) % N
        
        # (ab)_2 = a_2 + b_2 + a_2 b_0 + a_0 b_2 + G a_2 b_1 + H a_2 b_2 + I a_2 b_3
        r2 = (a2 + b2 + a2 * b0 + a0 * b2 + G * a2 * b1 + H * a2 * b2 + I * a2 * b3) % N

        # (ab)_3 = a_3 + b_3 + a_3 b_0 + a_0 b_3 + G a_3 b_1 + H a_3 b_2 + I a_3 b_3
        r3 = (a3 + b3 + a3 * b0 + a0 * b3 + G * a3 * b1 + H * a3 * b2 + I * a3 * b3) % N

        return M4Element([r0, r1, r2, r3], self.system)

    # Implements exponentiation a^n (repeated application of '*')
    # Uses exponentiation by squaring for efficiency
    def __pow__(self, exponent: int):
        if not isinstance(exponent, int) or exponent < 0:
            raise ValueError("Exponent must be a non-negative integer.")
        
        # The neutral element 'e' (multiplicative identity) as defined in the article (0,0,0,0)
        # a * e = e * a = a
        identity_element = M4Element([0, 0, 0, 0], self.system) 

        if exponent == 0:
            return identity_element
        
        # Start with the base vector 'a'
        base = self 
        # Initialize result with the identity element
        result = identity_element

        # Exponentiation by squaring algorithm
        while exponent > 0:
            if exponent % 2 == 1: # If the current bit of the exponent is 1
                result = result * base # Multiply result by the current base power
            base = base * base       # Square the base
            exponent //= 2           # Integer division by 2 (shift exponent right)

        return result

    # String representation for debugging and printing
    def __repr__(self):
        # Using id(self.system) to show that elements belong to the same system instance
        return f"M4Element(value={self.value}, system_id={id(self.system)})" 

    # User-friendly text representation
    def text(self):
        return str(self.value)

# --- Example Usage ---

# 1. Define the specific M4System with its parameters and modulus
#    These are the fixed parameters for your algebraic structure for K^4
my_system_4d = M4System(A=1, B=1, C=1, D=1, E=1, F=1, G=1, H=1, I=1, modulus=17)
print(f"Defined 4D System: {my_system_4d}\n")

# 2. Create M4Element instances within this system
a_4d = M4Element([1, 2, 3, 4], my_system_4d)
e_4d = M4Element([0, 0, 0, 0], my_system_4d) # The neutral element in this system

print(f"Base vector 'a' = {a_4d.text()}")
print(f"Neutral element 'e' = {e_4d.text()}")

# Test the neutral element property: a * e = a and e * a = a
print(f"a * e = {(a_4d * e_4d).text()}") # Should be [1, 2, 3, 4]
print(f"e * a = {(e_4d * a_4d).text()}\n") # Should be [1, 2, 3, 4]

# Test powers a^n
print(f"a^2 (a * a) = {(a_4d * a_4d).text()}")
print(f"a^2 (a**2)  = {(a_4d**2).text()}")
print(f"a^3 (a**3)  = {(a_4d**3).text()}")
print(f"a^4 (a**4)  = {(a_4d**4).text()}\n")

# Test power associativity: (a*a)*(a*a) == (a*a*a)*a == a^4
print(f"--- Test Power Associativity ---")
print(f"((a*a)*(a*a)) = {((a_4d*a_4d)*(a_4d*a_4d)).text()}") # Should be a^4
print(f"((a*a*a)*a) = {((a_4d*a_4d*a_4d)*a_4d).text()}")     # Should be a^4
print(f"a^4 from __pow__ = {(a_4d**4).text()}\n")

# Test internal commutativity: a^m * a^n = a^n * a^m = a^(m+n)
m_exp_4d = 2
n_exp_4d = 3
print(f"--- Test Internal Commutativity (m={m_exp_4d}, n={n_exp_4d}) ---")
print(f"a^m * a^n = {(a_4d**m_exp_4d * a_4d**n_exp_4d).text()}") # a^2 * a^3
print(f"a^n * a^m = {(a_4d**n_exp_4d * a_4d**m_exp_4d).text()}") # a^3 * a^2
print(f"a^(m+n)   = {(a_4d**(m_exp_4d + n_exp_4d)).text()}\n") # a^5

# Example of a Diffie-Hellman-like key exchange in 4D
print("--- Diffie-Hellman-like Key Exchange Example (4D) ---")
# Publicly agreed base vector (from the system)
public_a_4d = M4Element([5, 8, 12, 6], my_system_4d) 
print(f"Public base 'a': {public_a_4d.text()}")

# Alice's secret exponent
alice_secret_m_4d = 7
# Alice computes and sends A = a^m
A_alice_4d = public_a_4d ** alice_secret_m_4d
print(f"Alice's public A: {A_alice_4d.text()}")

# Bob's secret exponent
bob_secret_n_4d = 11
# Bob computes and sends B = a^n
B_bob_4d = public_a_4d ** bob_secret_n_4d
print(f"Bob's public B: {B_bob_4d.text()}")

# Alice computes shared key K_alice = B^m
K_alice_4d = B_bob_4d ** alice_secret_m_4d
print(f"Alice's computed key (B^m): {K_alice_4d.text()}")

# Bob computes shared key K_bob = A^n
K_bob_4d = A_alice_4d ** bob_secret_n_4d
print(f"Bob's computed key (A^n): {K_bob_4d.text()}")

# Verify if keys match (they should, due to power associativity and internal commutativity)
print(f"Keys match: {K_alice_4d.value == K_bob_4d.value}")
print(f"Expected key (a^(m*n)): {(public_a_4d ** (alice_secret_m_4d * bob_secret_n_4d)).text()}")

# Test with different parameters (create a new system instance)
print("\n--- Testing a different 4D system ---")
another_system_4d = M4System(A=2, B=3, C=4, D=5, E=6, F=7, G=8, H=9, I=10, modulus=19)
print(f"Defined System: {another_system_4d}")
b_4d = M4Element([1, 2, 3, 4], another_system_4d)
print(f"b = {b_4d.text()}")
print(f"b^2 = {(b_4d**2).text()}")

Defined 4D System: M4System(A=1, B=1, C=1, D=1, E=1, F=1, G=1, H=1, I=1, modulus=17)

Base vector 'a' = [1, 2, 3, 4]
Neutral element 'e' = [0, 0, 0, 0]
a * e = [1, 2, 3, 4]
e * a = [1, 2, 3, 4]

a^2 (a * a) = [7, 9, 5, 1]
a^2 (a**2)  = [7, 9, 5, 1]
a^3 (a**3)  = [16, 13, 11, 9]
a^4 (a**4)  = [8, 7, 2, 14]

--- Test Power Associativity ---
((a*a)*(a*a)) = [8, 7, 2, 14]
((a*a*a)*a) = [8, 7, 2, 14]
a^4 from __pow__ = [8, 7, 2, 14]

--- Test Internal Commutativity (m=2, n=3) ---
a^m * a^n = [14, 10, 15, 3]
a^n * a^m = [14, 10, 15, 3]
a^(m+n)   = [14, 10, 15, 3]

--- Diffie-Hellman-like Key Exchange Example (4D) ---
Public base 'a': [5, 8, 12, 6]
Alice's public A: [15, 14, 4, 2]
Bob's public B: [13, 12, 1, 9]
Alice's computed key (B^m): [12, 13, 11, 14]
Bob's computed key (A^n): [12, 13, 11, 14]
Keys match: True
Expected key (a^(m*n)): [12, 13, 11, 14]

--- Testing a different 4D system ---
Defined System: M4System(A=2, B=3, C=4, D=5, E=6, F=7, G=8, H=9, I=10, modulus=19)
b = [1, 2, 3, 4]
b